Implementation of AlexNEt, VGG16 and LeNet5 on CIFAR dataset

In [1]:
#Import the libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import matplotlib.pyplot as plt
import PIL
import seaborn as sns
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from IPython.display import display
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [3]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(50000, 32, 32, 3)
(50000, 1)
(10000, 32, 32, 3)
(10000, 1)


In [26]:
from keras.utils import to_categorical
#normalize
X_train = X_train.astype('float32')/255
X_test = X_test.astype('float32')/255

#categorical to one hot encoded

y_test = to_categorical(y_test, 10)
y_train = to_categorical(y_train, 10)


In [48]:
from keras.models import Sequential

alexnet=Sequential()

alexnet.add(Conv2D(filters=96,kernel_size=(11,11),strides=(4,4),padding="same",activation="relu",input_shape=(32,32,3)))
alexnet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

alexnet.add(Conv2D(filters=256,kernel_size=(11,11),strides=(1,1),padding="same",activation="relu"))
alexnet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

alexnet.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="same",activation="relu"))
alexnet.add(Conv2D(filters=384,kernel_size=(3,3),strides=(1,1),padding="same",activation="relu"))
alexnet.add(Conv2D(filters=256,kernel_size=(3,3),strides=(1,1),padding="same",activation="relu"))
alexnet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="valid"))

alexnet.add(Flatten())
alexnet.add(Dense(units=4096,input_shape=(32*32*3,),activation='relu'))
alexnet.add(Dropout(0.4))
alexnet.add(Dense(units=4096,activation='relu'))
alexnet.add(Dropout(0.4))
alexnet.add(Dense(units=1000,activation='relu'))
alexnet.add(Dropout(0.4))
alexnet.add(Dense(units=10,activation='softmax'))


In [49]:
alexnet.compile(loss = 'categorical_crossentropy', optimizer= 'adam' , metrics= ["accuracy"])

In [50]:
#using early stopping to exit training if validation loss is not decreasing even after certain epochs (patience)
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

#save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath="weights.hdf5", verbose=1, save_best_only=True)

In [54]:
history = alexnet.fit(X_train, y_train, epochs = 20, batch_size=32,steps_per_epoch=20)

Epoch 1/20
20/20 [==============================] - 16s 804ms/step - loss: 2.3009 - accuracy: 0.1187
Epoch 2/20
20/20 [==============================] - 16s 803ms/step - loss: 2.3032 - accuracy: 0.0906
Epoch 3/20
20/20 [==============================] - 16s 802ms/step - loss: 2.3026 - accuracy: 0.1094
Epoch 4/20
20/20 [==============================] - 16s 802ms/step - loss: 2.3031 - accuracy: 0.1141
Epoch 5/20
20/20 [==============================] - 16s 801ms/step - loss: 2.3020 - accuracy: 0.1000
Epoch 6/20
20/20 [==============================] - 16s 803ms/step - loss: 2.3041 - accuracy: 0.0953
Epoch 7/20
20/20 [==============================] - 16s 800ms/step - loss: 2.3020 - accuracy: 0.1016
Epoch 8/20
20/20 [==============================] - 16s 806ms/step - loss: 2.3050 - accuracy: 0.0812
Epoch 9/20
20/20 [==============================] - 16s 803ms/step - loss: 2.3023 - accuracy: 0.1016
Epoch 10/20
20/20 [==============================] - 16s 803ms/step - loss: 2.3015 - accura

In [55]:
alexnet.evaluate(X_test, y_test)

313/313 [==============================] - 35s 111ms/step - loss: 2.3027 - accuracy: 0.1000


[2.302729606628418, 0.10000000149011612]

LeNet5

In [31]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense

lenet=Sequential()
lenet.add(Conv2D(filters=6,kernel_size=(5,5),strides=(1,1),activation="sigmoid",input_shape=(32,32,3)))
lenet.add(MaxPooling2D(pool_size=(2,2)))
lenet.add(Conv2D(filters=16,kernel_size=(5,5),strides=(1,1),activation="sigmoid"))
lenet.add(MaxPooling2D(pool_size=(2,2)))
lenet.add(Flatten())
lenet.add(Dense(units=120,activation='tanh'))
lenet.add(Dense(units=84,activation='tanh'))
lenet.add(Dense(units=10,activation='softmax'))

lenet.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [33]:
history = lenet.fit(X_train,y_train,epochs=50,batch_size=32,steps_per_epoch=30,validation_split=0.2, callbacks=[checkpointer, earlystopping])

Epoch 1/50
30/30 [==============================] - ETA: 0s - loss: 2.3261 - accuracy: 0.0885
Epoch 00001: val_loss improved from inf to 2.31761, saving model to weights.hdf5
30/30 [==============================] - 4s 140ms/step - loss: 2.3261 - accuracy: 0.0885 - val_loss: 2.3176 - val_accuracy: 0.0952
Epoch 2/50
29/30 [============================>.] - ETA: 0s - loss: 2.3215 - accuracy: 0.1056
Epoch 00002: val_loss did not improve from 2.31761
30/30 [==============================] - 4s 138ms/step - loss: 2.3209 - accuracy: 0.1083 - val_loss: 2.3304 - val_accuracy: 0.0977
Epoch 3/50
30/30 [==============================] - ETA: 0s - loss: 2.3218 - accuracy: 0.0958
Epoch 00003: val_loss did not improve from 2.31761
30/30 [==============================] - 4s 138ms/step - loss: 2.3218 - accuracy: 0.0958 - val_loss: 2.3329 - val_accuracy: 0.1016
Epoch 4/50
29/30 [============================>.] - ETA: 0s - loss: 2.3275 - accuracy: 0.0808
Epoch 00004: val_loss did not improve from 2.317

In [37]:
lenet.evaluate(X_test, y_test,verbose=1)

313/313 [==============================] - 4s 12ms/step - loss: 2.3193 - accuracy: 0.1000


[2.3193037509918213, 0.10000000149011612]

VGG16

In [38]:
vgg16=Sequential()

vgg16.add(Conv2D(filters=64,kernel_size=(3,3),strides=(2,2),padding="same",activation="relu",input_shape=(32,32,3)))
vgg16.add(Conv2D(filters=64,kernel_size=(3,3),strides=(2,2),padding="same",activation="relu"))
vgg16.add(MaxPooling2D(pool_size=(2,2)))

vgg16.add(Conv2D(filters=128,kernel_size=(3,3),strides=(2,2),padding="same",activation="relu"))
vgg16.add(Conv2D(filters=128,kernel_size=(3,3),strides=(2,2),padding="same",activation="relu"))
vgg16.add(MaxPooling2D(pool_size=(2,2),padding="same"))

vgg16.add(Conv2D(filters=256,kernel_size=(3,3),strides=(2,2),padding="same",activation="relu"))
vgg16.add(Conv2D(filters=256,kernel_size=(3,3),strides=(2,2),padding="same",activation="relu"))
vgg16.add(Conv2D(filters=256,kernel_size=(3,3),strides=(2,2),padding="same",activation="relu"))
vgg16.add(MaxPooling2D(pool_size=(2,2),padding="same"))

vgg16.add(Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding="same",activation="relu"))
vgg16.add(Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding="same",activation="relu"))
vgg16.add(Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding="same",activation="relu"))
vgg16.add(MaxPooling2D(pool_size=(2,2),padding="same"))

vgg16.add(Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding="same",activation="relu"))
vgg16.add(Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding="same",activation="relu"))
vgg16.add(Conv2D(filters=512,kernel_size=(3,3),strides=(1,1),padding="same",activation="relu"))
vgg16.add(MaxPooling2D(pool_size=(2,2),padding="same"))
vgg16.add(Flatten())
vgg16.add(Dense(units=128,activation='relu'))
vgg16.add(Dense(units=10,activation='softmax'))

vgg16.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='sgd')

In [ ]:
#using early stopping to exit training if validation loss is not decreasing even after certain epochs (patience)
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

#save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath="weights.hdf5", verbose=1, save_best_only=True)

In [46]:
history = vgg16.fit(X_train,y_train,epochs=10,batch_size=32,steps_per_epoch=100,validation_split=0.2,verbose = 1)

Epoch 1/10
100/100 [==============================] - 131s 1s/step - loss: 2.3025 - accuracy: 0.1006 - val_loss: 2.3027 - val_accuracy: 0.0980
Epoch 2/10
100/100 [==============================] - 132s 1s/step - loss: 2.3024 - accuracy: 0.1016 - val_loss: 2.3028 - val_accuracy: 0.1003
Epoch 3/10
100/100 [==============================] - 132s 1s/step - loss: 2.3029 - accuracy: 0.0941 - val_loss: 2.3026 - val_accuracy: 0.1003
Epoch 4/10
100/100 [==============================] - 133s 1s/step - loss: 2.3028 - accuracy: 0.1016 - val_loss: 2.3026 - val_accuracy: 0.1022
Epoch 5/10
100/100 [==============================] - 132s 1s/step - loss: 2.3027 - accuracy: 0.1000 - val_loss: 2.3026 - val_accuracy: 0.1022
Epoch 6/10
100/100 [==============================] - 131s 1s/step - loss: 2.3028 - accuracy: 0.0931 - val_loss: 2.3026 - val_accuracy: 0.1022
Epoch 7/10
100/100 [==============================] - 132s 1s/step - loss: 2.3026 - accuracy: 0.1044 - val_loss: 2.3026 - val_accuracy: 0.1014

In [47]:
vgg16.evaluate(X_test, y_test,verbose=1)

313/313 [==============================] - 10s 32ms/step - loss: 2.3026 - accuracy: 0.1000


[2.30259370803833, 0.10000000149011612]